https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [1]:
import pandas as pd
data = pd.read_csv('df_processed.csv',error_bad_lines=False);
data = data.dropna(how='any',axis=0)
data_text = data[['Description']]
data_text['index'] = data_text.index
documents = data_text

/Users/sanjayshah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [2]:
documents[:10]

,Description,index
0,The Instant New York Times and USA Today Best...,0
1,"Phaedra Patrick's debut novel, The Curious Ch...",1
2,Whether it's selling your company's product i...,2
3,Christian Apologetics: An Anthology of Primar...,3
5,An instant New York Times Bestseller and Augu...,5
8,We’ve heard plenty from politicians and exper...,8
10,After serving out a year of hard labor in the...,10
14,Donald Trump won the presidency by being a on...,14
16,Grad student and cat lover Dulcie Schwartz m...,16
20,"Cook, eat, and be fit with 200 recipes from B...",20


In [3]:
documents['Description'].iloc[0]

' The Instant New York Times and USA Today Bestseller!When Washington D.C. was first built, it was on top of a swamp that had to be drained. Donald Trump says it\'s time to drain it again. In The Swamp, bestselling author Eric Bolling presents an infuriating, amusing, revealing, and outrageous history of American politics, past and present, Republican and Democrat. From national political scandals to tempests in a teapot that blew up; bribery, blackmail, bullying, and backroom deals that contradicted public policies; cronyism that cost taxpayers hundreds upon hundreds of millions of dollars; and personal conduct that can only be described as regrettable, The Swamp is a journey downriver through the bayous and marshes of Capitol Hill and Foggy Bottom. The presidential election of 2016 was ugly, but it exposed a political, media, industry, and elite establishment that desperately wanted to elect a politician who received millions of dollars from terror-funding states over a businessman w

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjayshah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
doc_sample = documents[documents['index'] == 1].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['', 'Part', 'road-trip', 'tale,', 'part', 'travelogue', 'of', 'lost', 'and', 'found', 'landscapes,', 'all', 'good-natured', 'natural', 'history,', 'Mariposa', 'Road', 'tracks', 'Bob', 'Pyle’s', 'journey', 'across', 'the', 'United', 'States', 'as', 'he', 'races', 'against', 'the', 'calendarin', 'his', 'search', 'for', 'as', 'many', 'of', 'the', '800', 'American', 'butterflies', 'as', 'he', 'can', 'find.\xa0Like', 'Pyle’s', 'classic', 'Chasing', 'Monarchs,', 'Mariposa', 'Road', 'recounts', 'his', 'adventures,', 'high', 'and', 'low,', 'in', 'tracking', 'down', 'butterflies', 'in', 'his', 'own', 'low-tech,', 'individual', 'way.', 'Accompanied', 'by', 'Marsha,', 'his', 'cottonwood-limb', 'butterfly', 'net;', 'Powdermilk,', 'his', '1982', 'Honda', 'Civic', 'with', '345,000', 'miles', 'on', 'the', 'odometer;', 'and', 'the', 'small', 'Leitz', 'binoculars', 'he', 'has', 'carried', 'for', 'more', 'than', 'thirty', 'years,', 'Bob', 'ventured', 'out', 'in', 'a', 'series', 'of'

In [6]:
processed_docs = documents['Description'].map(preprocess)

In [7]:
processed_docs = documents['Description'].map(preprocess)
processed_docs[:10]

0     [instant, york, time, today, bestsel, washingt...
1     [phaedra, patrick, debut, novel, curious, char...
2     [sell, compani, product, boardroom, sell, eat,...
3     [christian, apologet, antholog, primari, sourc...
5     [instant, york, time, bestsel, august, library...
8     [hear, plenti, politician, expert, affirm, act...
10    [serv, year, hard, labor, salt, mine, endovi, ...
14    [donald, trump, presid, wreck, ball, dysfunct,...
16    [grad, student, lover, dulci, schwartz, fight,...
20    [cook, recip, bobbi, flay, approach, healthi, ...
Name: Description, dtype: object

In [8]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [9]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 account
1 america
2 american
3 amus
4 author
5 backroom
6 bayous
7 bestsel
8 bill
9 blackmail
10 blow


In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 2),
 (6, 1),
 (7, 4),
 (8, 2),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 2),
 (24, 1),
 (25, 1),
 (26, 3),
 (27, 1),
 (28, 1),
 (29, 2),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 2),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 4),
 (53, 2),
 (54, 1),
 (55, 2),
 (56, 2),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 3),
 (74, 2),
 (75, 3),
 (76, 1),
 (77, 1),
 (78, 2),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1)]

In [12]:
bow_doc_0 = bow_corpus[0]

for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                                     dictionary[bow_doc_0[i][0]], 
                                                     bow_doc_0[i][1]))

Word 0 ("account") appears 1 time.
Word 1 ("america") appears 1 time.
Word 2 ("american") appears 1 time.
Word 3 ("amus") appears 1 time.
Word 4 ("author") appears 1 time.
Word 5 ("bestsel") appears 2 time.
Word 6 ("blow") appears 1 time.
Word 7 ("book") appears 4 time.
Word 8 ("build") appears 2 time.
Word 9 ("bulli") appears 1 time.
Word 10 ("climat") appears 1 time.
Word 11 ("concis") appears 1 time.
Word 12 ("conclud") appears 1 time.
Word 13 ("conduct") appears 1 time.
Word 14 ("contradict") appears 1 time.
Word 15 ("corrupt") appears 1 time.
Word 16 ("cost") appears 1 time.
Word 17 ("deal") appears 1 time.
Word 18 ("decis") appears 1 time.
Word 19 ("declar") appears 1 time.
Word 20 ("democrat") appears 1 time.
Word 21 ("describ") appears 1 time.
Word 22 ("desper") appears 1 time.
Word 23 ("dollar") appears 2 time.
Word 24 ("donald") appears 1 time.
Word 25 ("effect") appears 1 time.
Word 26 ("elect") appears 3 time.
Word 27 ("elit") appears 1 time.
Word 28 ("entertain") appears 1

In [13]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [14]:
corpus_tfidf = tfidf[bow_corpus]

In [18]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.1896817565175649),
 (1, 0.13616465623012936),
 (2, 0.1815469917216657),
 (3, 0.1777799590017366),
 (4, 0.19877564986109572),
 (5, 0.1777799590017366),
 (6, 0.1777799590017366),
 (7, 0.17075613215210664),
 (8, 0.14770983022422038),
 (9, 0.15837625970208172),
 (10, 0.13616465623012936),
 (11, 0.3793635130351298),
 (12, 0.1896817565175649),
 (13, 0.11893599809069934),
 (14, 0.1777799590017366),
 (15, 0.1940948922144773),
 (16, 0.3793635130351298),
 (17, 0.16747015304561255),
 (18, 0.13616465623012936),
 (19, 0.13616465623012936),
 (20, 0.20375859519361803),
 (21, 0.18550728826012086),
 (22, 0.1896817565175649),
 (23, 0.1815469917216657),
 (24, 0.1815469917216657),
 (25, 0.16129014722003585),
 (26, 0.08593353504918798)]


In [15]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [16]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.015*"book" + 0.010*"life" + 0.008*"time" + 0.008*"love" + 0.007*"famili" + 0.006*"world" + 0.006*"stori" + 0.006*"year" + 0.005*"live" + 0.005*"best"
Topic: 1 
Words: 0.010*"book" + 0.009*"world" + 0.008*"time" + 0.008*"life" + 0.005*"author" + 0.005*"live" + 0.005*"peopl" + 0.005*"learn" + 0.005*"help" + 0.004*"love"
Topic: 2 
Words: 0.016*"book" + 0.006*"work" + 0.006*"time" + 0.006*"world" + 0.006*"year" + 0.006*"life" + 0.006*"stori" + 0.005*"busi" + 0.005*"chang" + 0.004*"american"
Topic: 3 
Words: 0.011*"world" + 0.009*"life" + 0.009*"time" + 0.006*"peopl" + 0.006*"know" + 0.005*"year" + 0.005*"help" + 0.005*"like" + 0.005*"stori" + 0.004*"work"
Topic: 4 
Words: 0.013*"book" + 0.008*"stori" + 0.007*"time" + 0.006*"author" + 0.005*"year" + 0.004*"read" + 0.004*"world" + 0.004*"live" + 0.004*"star" + 0.004*"american"
Topic: 5 
Words: 0.008*"life" + 0.008*"stori" + 0.008*"time" + 0.008*"know" + 0.007*"year" + 0.006*"love" + 0.006*"come" + 0.005*"like" + 0.005*"fri

In [17]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [18]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.003*"book" + 0.003*"life" + 0.003*"color" + 0.002*"think" + 0.002*"love" + 0.002*"live" + 0.002*"time" + 0.002*"stori" + 0.002*"world" + 0.002*"help"
Topic: 1 Word: 0.006*"book" + 0.002*"color" + 0.002*"life" + 0.002*"health" + 0.002*"illustr" + 0.002*"help" + 0.002*"famili" + 0.002*"world" + 0.002*"design" + 0.002*"chang"
Topic: 2 Word: 0.003*"book" + 0.002*"stori" + 0.002*"world" + 0.002*"life" + 0.002*"like" + 0.002*"live" + 0.002*"time" + 0.002*"know" + 0.002*"histori" + 0.002*"american"
Topic: 3 Word: 0.002*"book" + 0.002*"world" + 0.002*"stori" + 0.002*"life" + 0.002*"love" + 0.002*"polit" + 0.002*"live" + 0.002*"year" + 0.002*"time" + 0.002*"work"
Topic: 4 Word: 0.003*"book" + 0.002*"love" + 0.002*"children" + 0.002*"stori" + 0.002*"life" + 0.002*"mother" + 0.002*"heart" + 0.002*"author" + 0.002*"world" + 0.002*"famili"
Topic: 5 Word: 0.002*"book" + 0.002*"game" + 0.002*"life" + 0.002*"stori" + 0.002*"live" + 0.002*"time" + 0.002*"world" + 0.002*"famili" + 0.002

In [19]:
processed_docs[1]

['phaedra',
 'patrick',
 'debut',
 'novel',
 'curious',
 'charm',
 'arthur',
 'pepper',
 'hail',
 'poignant',
 'utter',
 'endear',
 'return',
 'rise',
 'shine',
 'benedict',
 'stone',
 'novel',
 'famili',
 'forgiv',
 'second',
 'chanc',
 'happi',
 'moonston',
 'empathi',
 'azurit',
 'memori',
 'lapi',
 'lazuli',
 'truth',
 'quiet',
 'villag',
 'noon',
 'benedict',
 'stone',
 'settl',
 'complac',
 'predict',
 'routin',
 'busi',
 'jewelri',
 'shop',
 'dri',
 'marriag',
 'rock',
 'life',
 'desper',
 'need',
 'jump',
 'start',
 'surpris',
 'arriv',
 'door',
 'gemma',
 'benedict',
 'audaci',
 'teenag',
 'niec',
 'daughter',
 'estrang',
 'brother',
 'charli',
 'stone',
 'brother',
 'fall',
 'haven',
 'speak',
 'decad',
 'charli',
 'leav',
 'america',
 'reckless',
 'stubborn',
 'gemma',
 'invit',
 'benedict',
 'world',
 'turn',
 'order',
 'life',
 'upsid',
 'exact',
 'need',
 'life',
 'track',
 'fill',
 'color',
 'charact',
 'irresist',
 'charm',
 'rise',
 'shine',
 'benedict',
 'stone',
 'lu

In [20]:
for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7912186980247498	 
Topic: 0.008*"life" + 0.008*"stori" + 0.008*"time" + 0.008*"know" + 0.007*"year" + 0.006*"love" + 0.006*"come" + 0.005*"like" + 0.005*"friend" + 0.005*"world"

Score: 0.19890256226062775	 
Topic: 0.015*"book" + 0.010*"life" + 0.008*"time" + 0.008*"love" + 0.007*"famili" + 0.006*"world" + 0.006*"stori" + 0.006*"year" + 0.005*"live" + 0.005*"best"


In [21]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.3969144821166992	 
Topic: 0.003*"stori" + 0.002*"world" + 0.002*"book" + 0.002*"life" + 0.002*"time" + 0.002*"busi" + 0.002*"year" + 0.002*"best" + 0.002*"leav" + 0.002*"love"

Score: 0.3961485028266907	 
Topic: 0.002*"book" + 0.002*"world" + 0.002*"stori" + 0.002*"life" + 0.002*"love" + 0.002*"polit" + 0.002*"live" + 0.002*"year" + 0.002*"time" + 0.002*"work"

Score: 0.19829292595386505	 
Topic: 0.003*"book" + 0.002*"love" + 0.002*"life" + 0.002*"time" + 0.002*"stori" + 0.002*"work" + 0.002*"world" + 0.002*"york" + 0.002*"live" + 0.002*"peopl"
